# Project Echo - Experiment Benchmarking Framework

This notebook provides an interactive interface to the benchmarking framework. It allows you to run various experiments with different model architectures and augmentation strategies, and compare their performance.

## Overview

The benchmarking framework is designed to systematically evaluate different combinations of:
- Model architectures (EfficientNet, MobileNet, ResNet, etc.)
- Audio augmentation strategies
- Image/spectrogram augmentation strategies

Results are collected and visualized to help identify the best performing configurations for bat sound classification.

## 1. Import Required Libraries

In [1]:
# Import necessary libraries
import os
import sys
import json
import datetime
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from ipywidgets import widgets
from IPython.display import display, HTML, clear_output
import re
import importlib
import config.system_config # To reload the module

# Add the current directory to path for imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import framework components
from config.experiment_configs import EXPERIMENTS





## 2. Configuration

Set up the directories and options for benchmarking.

In [2]:
# Default directories - you can change these as needed
DATA_DIR = r"C:\Users\deanf\OneDrive\CTMC\MAppAI\SIT764 Team Project A\Project Echo"  # Directory containing audio data
CACHE_DIR = r"C:\Users\deanf\OneDrive\CTMC\MAppAI\SIT764 Team Project A\pipeline_cache"  # Directory for caching pipeline results
OUTPUT_DIR = r"./results"  # Directory to save experiment results

# Create output directory if it doesn't exist
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Configure GPU memory if available
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"GPU support enabled: {len(gpus)} GPU(s) found")
else:
    print("No GPU support found, running on CPU")



No GPU support found, running on CPU


## 3. Available Experiments

Here you can view and select experiments to run. Each experiment represents a combination of model architecture and augmentation strategies.

In [3]:
# Display available experiments
experiment_data = []
for exp in EXPERIMENTS:
    experiment_data.append({
        "name": exp["name"],
        "model": exp["model"],
        "audio_augmentation": exp["audio_augmentation"],
        "image_augmentation": exp["image_augmentation"],
        "epochs": exp["epochs"],
        "batch_size": exp["batch_size"]
    })

experiments_df = pd.DataFrame(experiment_data)
display(experiments_df)

,name,model,audio_augmentation,image_augmentation,epochs,batch_size
0,baseline,EfficientNetV2B0,none,none,10,16
1,basic_audio_aug,EfficientNetV2B0,basic,none,10,16
2,basic_image_aug,EfficientNetV2B0,none,basic_rotation,10,16
3,full_augmentation,EfficientNetV2B0,advanced,combined,10,16
4,mobilenet_baseline,MobileNetV2,none,none,10,16
5,mobilenet_full_aug,MobileNetV2,advanced,combined,10,16


## 4. Interactive Experiment Selection

Use the widgets below to select experiments and set directories.

In [4]:
# Create widgets for directory selection
data_dir_widget = widgets.Text(
    value=DATA_DIR,
    description='Data Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

cache_dir_widget = widgets.Text(
    value=CACHE_DIR,
    description='Cache Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

output_dir_widget = widgets.Text(
    value=OUTPUT_DIR,
    description='Output Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Create widget for experiment selection
experiment_options = [(exp["name"], exp["name"]) for exp in EXPERIMENTS]
experiment_widget = widgets.SelectMultiple(
    options=experiment_options,
    description='Select Experiments:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', height='200px')
)

# Buttons for actions
run_selected_button = widgets.Button(
    description='Run Selected Experiments',
    button_style='primary',
    tooltip='Run the selected experiments'
)

run_all_button = widgets.Button(
    description='Run All Experiments',
    tooltip='Run all experiments'
)

generate_report_button = widgets.Button(
    description='Generate Report Only',
    button_style='info',
    tooltip='Generate a report from existing results'
)

# Output area for logs
output_area = widgets.Output(layout={'border': '1px solid black'})

# Display widgets
display(data_dir_widget)
display(cache_dir_widget)
display(output_dir_widget)
display(experiment_widget)
display(widgets.HBox([run_selected_button, run_all_button, generate_report_button]))
display(output_area)

Text(value='C:\\Users\\deanf\\OneDrive\\CTMC\\MAppAI\\SIT764 Team Project A\\Project Echo', description='Data …

Text(value='C:\\Users\\deanf\\OneDrive\\CTMC\\MAppAI\\SIT764 Team Project A\\pipeline_cache', description='Cac…

Text(value='./results', description='Output Directory:', layout=Layout(width='80%'), style=TextStyle(descripti…

SelectMultiple(description='Select Experiments:', layout=Layout(height='200px', width='50%'), options=(('basel…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

## 5. Experiment Runner Functions

These functions handle the execution of experiments and report generation.

In [6]:
from utils.optimised_engine_pipeline import train_model


def run_selected_experiments(b):
    from IPython.display import clear_output # Moved import here for clarity
    clear_output(wait=True) # Clear previous output first
    with output_area:
        print("Starting experiment run...")
        # Get new directory paths from widgets
        new_data_dir = data_dir_widget.value
        new_cache_dir = cache_dir_widget.value
        
        # Define path to system_config.py (relative to notebook directory)
        # Assumes 'config' is a subdirectory of the notebook's directory
        config_file_path = os.path.join('config', 'system_config.py')
        
        try:
            print(f"Attempting to update {config_file_path}...")
            with open(config_file_path, 'r') as f:
                lines = f.readlines()
            
            new_lines = []
            config_updated = False
            for line in lines:
                if "'AUDIO_DATA_DIRECTORY':" in line:
                    # Use r-string for replacement to handle backslashes in path correctly
                    new_line = re.sub(r"('AUDIO_DATA_DIRECTORY':\s*r\")[^\"]*(\")", rf'\1{new_data_dir}\2', line)
                    if new_line != line:
                        config_updated = True
                    new_lines.append(new_line)
                elif "'CACHE_DIRETORY':" in line: # Note the typo 'DIRETORY'
                    new_line = re.sub(r"('CACHE_DIRETORY':\s*r\")[^\"]*(\")", rf'\1{new_cache_dir}\2', line)
                    if new_line != line:
                        config_updated = True
                    new_lines.append(new_line)
                else:
                    new_lines.append(line)
            
            if config_updated:
                with open(config_file_path, 'w') as f:
                    f.writelines(new_lines)
                print(f"Successfully updated {config_file_path} with new directory paths.")
            else:
                print(f"{config_file_path} already up-to-date or keys not found.")
            
            # Reload the system_config module to apply changes
            importlib.reload(config.system_config)
            # Re-import SC if it's used directly in this notebook, or ensure train_model gets the fresh one.
            # from config.system_config import SC 
            print("System configuration reloaded.")
            
        except Exception as e:
            print(f"Error updating or reloading system_config.py: {e}")
            print("Proceeding with potentially outdated configuration.")
            # Decide if you want to return or proceed if config update fails
            # return 

        selected_experiments = list(experiment_widget.value)
        if not selected_experiments:
            print("No experiment selected. Please select at least one experiment.")
            return
        
        for exp_name in selected_experiments:
            exp_config = next((exp for exp in EXPERIMENTS if exp["name"] == exp_name), None)
            if exp_config is None:
                print(f"Experiment {exp_name} not found in EXPERIMENTS.")
                continue

            print(f"Running experiment: {exp_config['name']}")
            # Pass configuration values to the train_model function.
            # train_model will use the reloaded system_config.SC for DATA_DIR and CACHE_DIR
            model, history = train_model(
                model_name=exp_config['model'],
                epochs=exp_config.get('epochs'),
                batch_size=exp_config.get('batch_size')
            )
            print(f"Training completed for experiment: {exp_config['name']}")
            if model:
                 model.summary(print_fn=lambda x: print(x)) # Ensure summary prints to output_area
            print("-" * 40)

run_selected_button.on_click(run_selected_experiments)

## 6. View Previous Results

If you've already run experiments, you can view and analyze the results here.

In [7]:
# Under development
def load_results(output_dir=OUTPUT_DIR):
    # Check if results directory exists
    if not os.path.exists(output_dir):
        print(f"Results directory does not exist: {output_dir}")
        return None
    
    # Look for comparison report CSV
    csv_files = [f for f in os.listdir(output_dir) if f.startswith("comparison_results_") and f.endswith(".csv")]
    
    if not csv_files:
        print("No comparison results found. Run experiments or generate a report first.")
        return None
    
    # Load the latest CSV file
    latest_csv = max(csv_files)
    csv_path = os.path.join(output_dir, latest_csv)
    results_df = pd.read_csv(csv_path)
    
    print(f"Loaded results from: {csv_path}")
    return results_df

# Load and display results if available
results_df = load_results()
if results_df is not None:
    display(results_df)

No comparison results found. Run experiments or generate a report first.


## 7. Visualize Results

Create various visualizations to compare experiment results.

In [ ]:
# Taken from previously developed notebooks in Machine Learing course

def visualize_results(results_df):
    if results_df is None or len(results_df) == 0:
        print("No results available to visualize.")
        return
    
    # Set the figure size for better visibility
    plt.figure(figsize=(14, 8))
    
    # Create accuracy comparison bar chart
    plt.subplot(2, 2, 1)
    sns.barplot(x='Experiment', y='Test Accuracy', data=results_df)
    plt.title('Test Accuracy by Experiment')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Create F1 score comparison bar chart
    plt.subplot(2, 2, 2)
    sns.barplot(x='Experiment', y='F1 Score', data=results_df)
    plt.title('F1 Score by Experiment')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Training time comparison
    plt.subplot(2, 2, 3)
    sns.barplot(x='Experiment', y='Training Time (min)', data=results_df)
    plt.title('Training Time by Experiment (minutes)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Model comparison
    plt.subplot(2, 2, 4)
    model_comparison = results_df.groupby('Model')['Test Accuracy'].mean().reset_index()
    sns.barplot(x='Model', y='Test Accuracy', data=model_comparison)
    plt.title('Average Accuracy by Model')
    plt.tight_layout()
    
    plt.tight_layout(pad=3.0)
    plt.show()
    
    # Create a separate visualization for augmentation impact
    plt.figure(figsize=(12, 6))
    
    # Reshape data for augmentation comparison
    # The following code is yet to be tested and may need adjustments based on the actual DataFrame structure
    """
    aug_data = []
    for _, row in results_df.iterrows():
        aug_data.append({
            'Experiment': row['Experiment'],
            'Augmentation Type': 'Audio',
            'Augmentation': row['Audio Augmentation'],
            'Accuracy': row['Test Accuracy']
        })
        aug_data.append({
            'Experiment': row['Experiment'],
            'Augmentation Type': 'Image',
            'Augmentation': row['Image Augmentation'],
            'Accuracy': row['Test Accuracy']
        })"""
    
    aug_df = pd.DataFrame(aug_data)
    sns.barplot(x='Augmentation', y='Accuracy', hue='Augmentation Type', data=aug_df)
    plt.title('Accuracy by Augmentation Type')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Visualize results if available
if results_df is not None:
    visualize_results(results_df)

## 8. Experiment Analysis and Conclusions
(if required)


